In [40]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import torch.nn as nn
import torch

In [41]:
import numpy as np

n = 14  # Replace with the desired value of n

permutation = [np.random.permutation(n) for _ in range(3)]

print(permutation)


[array([ 7, 10,  1, 13,  9,  4,  3,  6,  0,  5, 12, 11,  8,  2]), array([13,  3,  8,  5,  9,  0,  7, 11, 12,  2,  4,  1,  6, 10]), array([ 7, 13,  4,  0,  9,  6,  5,  8, 12, 11, 10,  2,  1,  3])]


In [42]:
permutation[1]

array([13,  3,  8,  5,  9,  0,  7, 11, 12,  2,  4,  1,  6, 10])

### prepare dataset

In [43]:
# speculate how the data is constructed
burma_path = "./dataset/burma14.xml"
brazil_path = "./dataset/brazil58.xml"

tree_burma = ET.parse(burma_path)
root = tree_burma.getroot()

for i, e in enumerate(root):
    if e.text is None:
        text = "-"
    else:
        text = e.text
    print(i+1,e.tag,":",text)

1 name : burma14
2 source : TSPLIB
3 description : 14-Staedte in Burma (Zaw Win)
4 doublePrecision : 15
5 ignoredDigits : 5
6 graph : 
    


In [44]:
root.find('name').text

'burma14'

In [45]:
identity_matrix = np.identity(2)
result = identity_matrix * -np.inf
np.nan_to_num(result,0)

C:\Users\USER\AppData\Local\Temp\ipykernel_23396\4080773743.py:2: RuntimeWarning: invalid value encountered in multiply
  result = identity_matrix * -np.inf


array([[-1.79769313e+308,  0.00000000e+000],
       [ 0.00000000e+000, -1.79769313e+308]])

In [46]:
# function to load weight metrics
def load_metrics(xml_path, info=False):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # meta-data
    name = root.find('name').text
    source = root.find('source').text
    description = root.find('description').text
    doublePrecision = int(root.find('doublePrecision').text)
    ignoredDigits = int(root.find('ignoredDigits').text)
    num_node = len(root.findall(".//vertex"))
    weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)
    # weights_metric = np.identity(num_node)

    for i, vertex in enumerate(root.findall('.//vertex')):
        for edge in vertex.findall('.//edge'):
            cost = float(edge.get("cost"))
            node = int(edge.text)
            # print(f"line:{i} node:{node}->cost:{cost}")
            # if i == node:
            #     weights_metric[i,node] = -np.Inf
            # else:
            weights_metric[i,node] = cost
    if info is True:
        return weights_metric , (name, source, description, doublePrecision, ignoredDigits)

    return weights_metric




In [47]:
burma_metric, _ = load_metrics(burma_path,True)
burma_metric.shape

C:\Users\USER\AppData\Local\Temp\ipykernel_23396\2035547776.py:12: RuntimeWarning: invalid value encountered in multiply
  weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)


(14, 14)

In [48]:
_

('burma14', 'TSPLIB', '14-Staedte in Burma (Zaw Win)', 15, 5)

In [49]:
brazil_metric = load_metrics(brazil_path)
brazil_metric.shape

C:\Users\USER\AppData\Local\Temp\ipykernel_23396\2035547776.py:12: RuntimeWarning: invalid value encountered in multiply
  weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)


(58, 58)

### The Evolutionary Algorithm 

In [50]:
tournament_size = 10


$$
\text{cost} = \sum_{i=1}^{n-1} D[C[i]][C[i + 1]] + D[C[n]][C[1]]
$$



In [51]:
def cost_function(input):
    pass
    
def target_function(input) -> float | int:
    # TODO : implement cost function to evaluate the fitness of each candidate
    n = len(input)
    pass

def fitness_function(input, num_iter:int=20) -> float:
    pass

In [52]:
def generate_population(distance_metric, population_size):
    return [list(np.random.permutation(len(distance_metric))) for _ in range(population_size)]

In [53]:
def distance(candidate, distance_metric):
    total_distance = 0
    for i in range(len(candidate)-1):
        total_distance += distance_metric[candidate[i],candidate[i+1]]
    total_distance += distance_metric[candidate[len(candidate)-1],candidate[0]]
    return total_distance

In [54]:
pop_gen1 = generate_population(burma_metric, 10)
pop_gen1 

[[6, 11, 8, 2, 12, 5, 7, 10, 13, 9, 1, 3, 4, 0],
 [11, 0, 7, 8, 2, 10, 13, 5, 4, 9, 6, 1, 12, 3],
 [4, 12, 10, 3, 9, 13, 1, 7, 0, 8, 11, 5, 2, 6],
 [11, 5, 1, 9, 8, 0, 12, 13, 2, 4, 7, 6, 10, 3],
 [12, 10, 1, 13, 6, 2, 11, 3, 7, 8, 9, 0, 4, 5],
 [13, 10, 2, 11, 6, 7, 3, 1, 4, 9, 12, 0, 8, 5],
 [6, 1, 7, 5, 12, 9, 10, 2, 13, 3, 8, 11, 4, 0],
 [5, 3, 4, 12, 10, 13, 2, 8, 9, 7, 11, 1, 0, 6],
 [9, 7, 8, 4, 0, 1, 10, 2, 5, 12, 3, 6, 13, 11],
 [0, 3, 12, 6, 4, 10, 9, 8, 1, 7, 11, 2, 13, 5]]

In [55]:
distance = [ distance(gen, distance_metric=burma_metric) for gen in pop_gen1]
distance

[7105.0,
 6857.0,
 6335.0,
 5833.0,
 5419.0,
 7602.0,
 6866.0,
 5573.0,
 6531.0,
 5805.0]

In [56]:
np.zeros((10,10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<b>1. Generate an initial population of p randomly created solutions and assess the fitness of each individual in
the population.

<b>2. Use tournament selection twice to select two parents, denoted as a and b.


<b>3. apply a single-point crossover on these selected parents to generate two children, referred to as c and d.


In [57]:
locus = np.random.randint(low=0, high=14)
locus

6

In [58]:
def single_crossover(candidate_a, candidate_b):
    

SyntaxError: incomplete input (343184927.py, line 2)

<b>4. Run a mutation on c and d to give two new solutions e and f. Evaluate the fitness of e and f.

<b>5. Run replacement function, firstly for e, then f.


<b>6. If a termination criterion has been reached, then stop. Otherwise return to step 2.

### Burma14

In [ ]:
burma_path = "./dataset/burma14.xml"

tree_burma = ET.parse(burma_path)
root = tree_burma.getroot()

In [ ]:
text = ""
for i, e in enumerate(root):
    if e.text is None:
        text = "-"
    else:
        text = e.text
    print(i+1,e.tag,":",text)

1 name : burma14
2 source : TSPLIB
3 description : 14-Staedte in Burma (Zaw Win)
4 doublePrecision : 15
5 ignoredDigits : 5
6 graph : 
    


In [ ]:
doublePrecision = int(root.find('doublePrecision').text)
ignoredDigits = int(root.find('ignoredDigits').text)
tol = 10**(-(doublePrecision - ignoredDigits))
tol


1e-10

In [ ]:
num_burma_cities = len(root.findall(".//vertex"))
num_burma_cities

14

In [ ]:
graph_element = root.find('graph')

In [ ]:
vertex_elements = graph_element.findall('vertex')

# Get the length of the <vertex> tags
vertex_count = len(vertex_elements)

print(f"Number of <vertex> tags: {vertex_count}")

Number of <vertex> tags: 14


In [ ]:
lam = []

In [ ]:
lam = np.zeros((14,14))
lam.shape

(14, 14)

In [ ]:
float(1.530000000000000e+02)

153.0

In [ ]:
c = 0
for i , v in enumerate(graph_element):
    for e in v:
        lam[i,int(e.text)] = float(e.get("cost"))

In [ ]:
lam

array([[   0.,  153.,  510.,  706.,  966.,  581.,  455.,   70.,  160.,
         372.,  157.,  567.,  342.,  398.],
       [ 153.,    0.,  422.,  664.,  997.,  598.,  507.,  197.,  311.,
         479.,  310.,  581.,  417.,  376.],
       [ 510.,  422.,    0.,  289.,  744.,  390.,  437.,  491.,  645.,
         880.,  618.,  374.,  455.,  211.],
       [ 706.,  664.,  289.,    0.,  491.,  265.,  410.,  664.,  804.,
        1070.,  768.,  259.,  499.,  310.],
       [ 966.,  997.,  744.,  491.,    0.,  400.,  514.,  902.,  990.,
        1261.,  947.,  418.,  635.,  636.],
       [ 581.,  598.,  390.,  265.,  400.,    0.,  168.,  522.,  634.,
         910.,  593.,   19.,  284.,  239.],
       [ 455.,  507.,  437.,  410.,  514.,  168.,    0.,  389.,  482.,
         757.,  439.,  163.,  124.,  232.],
       [  70.,  197.,  491.,  664.,  902.,  522.,  389.,    0.,  154.,
         406.,  133.,  508.,  273.,  355.],
       [ 160.,  311.,  645.,  804.,  990.,  634.,  482.,  154.,    0.,
         

### Brazil58

In [ ]:
brazil_path = "./dataset/brazil58.xml"

tree_brazil = ET.parse(brazil_path)
root = tree_brazil.getroot()

In [ ]:
for i, e in enumerate(root):
    if e.text is None:
        text = "-"
    else:
        text = e.text
    print(i+1,e.tag,":",text)

1 name : brazil58
2 source : TSPLIB
3 description : 58 cities in Brazil (Ferreira)
4 doublePrecision : 15
5 ignoredDigits : 5
6 graph : 
    


In [ ]:
doublePrecision = int(root.find('doublePrecision').text)
ignoredDigits = int(root.find('ignoredDigits').text)
tol = 10**(-(doublePrecision - ignoredDigits))
tol

1e-10

In [ ]:
import math
edges = root.findall(".//edge")
math.sqrt(len(edges))

57.4978260458602

In [ ]:
# Find the 'edge' elements and extract the 'cost' attribute
edges = root.findall(".//edge")
weights = [float(edge.get("cost")) for edge in edges]

# Define the number of cities
num_cities = len(weights)
num_cities
# # Create a NumPy array to store the weights
# weights_array = np.zeros((num_cities, num_cities))

# # Populate the NumPy array with the edge costs
# for i in range(num_cities):
#     for j in range(num_cities):
#         if i != j:
#             weights_array[i, j] = weights[i * num_cities + j]

# # Print the NumPy array
# print(weights_array)

3306

In [ ]:
# import xml.etree.ElementTree as ET

# # Load the XML file
# tree = ET.parse('./dataset/brazil58.xml')
# root = tree.getroot()

# # Access elements in the XML
# name = root.find('name').text
# source = root.find('source').text
# description = root.find('description').text

# print("Name:", name)
# print("Source:", source)
# print("Description:", description)

# # Access vertex and edge data
# for vertex in root.findall('.//vertex'):
#     vertex_id = vertex.find('edge').text
#     edges = vertex.findall('edge')
    
#     print(f"Vertex ID: {vertex_id}")
#     for edge in edges:
#         cost = edge.get('cost')
#         destination = edge.text
#         print(f"Edge - Destination: {destination}, Cost: {cost}")


In [ ]:
# num_cities = len(root.findall(".//vertex"))
# num_cities

In [ ]:
graph_element = root.find('graph')

In [ ]:
vertex_element = graph_element.findall('vertex')

In [ ]:
for edge in vertex_element:
    print(f"city:[{(edge.text)}]-> cost:{(edge.get('cost'))}")
    

city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
city:[
      ]-> cost:None
c

In [ ]:
vertex_elements = graph_element.findall('vertex')

# Get the length of the <vertex> tags
vertex_count = len(vertex_elements)

print(f"Number of <vertex> tags: {vertex_count}")

Number of <vertex> tags: 58


In [ ]:
brazil_matrix = []
for i , v in enumerate(graph_element):
    print(v.tag)
        # brazil_matrix[i,int(e.text)] = float(e.get("cost"))


vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
vertex
